In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import glob
import os
#pd.set_option('display.max_rows', 200)

In [2]:
data = pd.read_excel('..\\Datos Partidos\\Sarmiento_Situaciones.xlsx')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Session Start Date   52 non-null     datetime64[ns]
 1   Event                52 non-null     object        
 2   Session Name         52 non-null     object        
 3   Session Start        52 non-null     object        
 4   Session End          52 non-null     object        
 5   Tag Description      52 non-null     object        
 6   Tag Notes            0 non-null      float64       
 7   Tag Start            52 non-null     object        
 8   Tag End              52 non-null     object        
 9   Tag Duration (secs)  52 non-null     float64       
 10  Categoria            52 non-null     object        
 11  Zona2                52 non-null     object        
 12  Zona                 52 non-null     object        
 13  Accion               52 non-null     

In [3]:
data.head()

,Session Start Date,Event,Session Name,Session Start,Session End,Tag Description,Tag Notes,Tag Start,Tag End,Tag Duration (secs),Categoria,Zona2,Zona,Accion,Jugada
0,2022-06-19,Sarmiento,Situaciones,15:25:20,17:40:10,A FAVOR,NaN,00:21:17,00:21:19,2000.0,ATAQUE DIRECTO,6.0,3.0,VOLVER,1.0
1,2022-06-19,Sarmiento,Situaciones,15:25:20,17:40:10,A FAVOR,NaN,00:21:25,00:21:27,2000.0,ATAQUE DIRECTO,8A,6.0,VOLVER,1.0
2,2022-06-19,Sarmiento,Situaciones,15:25:20,17:40:10,A FAVOR,NaN,00:21:33,00:21:35,2000.0,ATAQUE DIRECTO,7.0,8A,VOLVER,1.0
3,2022-06-19,Sarmiento,Situaciones,15:25:20,17:40:10,A FAVOR,NaN,00:21:39,00:21:41,2000.0,ATAQUE DIRECTO,8A,7.0,VOLVER,1.0
4,2022-06-19,Sarmiento,Situaciones,15:25:20,17:40:10,A FAVOR,NaN,00:21:45,00:21:47,2000.0,ATAQUE DIRECTO,8B,8A,REMATE AFUERA,1.0


In [4]:
data.columns = ['Fecha','etiquetado','name','Session Start','Session End','Evento','Tag Notes','Tag Start','Tag End','Tag Duration (secs)','Categoria','Zona inicial','Zona Final','Accion','Jugada']

In [5]:
'''CORREGIR MAYUSCULAS Y COSAS DE EVENTOS'''

data['Evento'] = data['Evento'].str.lower().str.strip()
data['Categoria'] = data['Categoria'].str.lower().str.strip()
data['Accion'] = data['Accion'].str.lower().str.strip()

## Estadisticas 

In [6]:
data.groupby(['Evento','Categoria'])['Jugada'].nunique().to_frame('Cantidad')

Cantidad
Evento    Categoria                 
a favor   ataque combinado         1
          ataque directo           2
          pelota parada            3
          transicion               3
en contra ataque combinado         2
          ataque directo           1
          pelota parada            2
          transicion               2

### Armo las coordenadas

In [7]:
data['Zona inicial'] = data['Zona inicial'].replace('2A',2.1,regex=True)
data['Zona inicial'] = data['Zona inicial'].replace('2B',2.2,regex=True)
data['Zona inicial'] = data['Zona inicial'].replace('8A',8.1,regex=True)
data['Zona inicial'] = data['Zona inicial'].replace('8B',8.2,regex=True)
data['Zona Final'] = data['Zona Final'].replace('2A',2.1,regex=True)
data['Zona Final'] = data['Zona Final'].replace('2B',2.2,regex=True)
data['Zona Final'] = data['Zona Final'].replace('8A',8.1,regex=True)
data['Zona Final'] = data['Zona Final'].replace('8B',8.2,regex=True)

In [8]:
data['Zona inicial'].value_counts()

8.2    13
2.2     7
4.0     6
6.0     5
5.0     5
8.1     4
7.0     4
2.1     3
9.0     2
3.0     2
1.0     1
Name: Zona inicial, dtype: int64

In [9]:
data['Zona Final'].value_counts()

6.0    8
4.0    8
5.0    7
7.0    6
3.0    5
8.1    4
1.0    4
2.1    4
8.2    4
9.0    2
Name: Zona Final, dtype: int64

In [10]:
map_zona_coord_x = {  
1:45,
2.1:70,
2.2:30,
3:45,
4:136,
5:136,
6:136,
7:227,
8.1:255,
8.2:207,
9:227
}

In [11]:
map_zona_coord_y = {  
1:135,
2.1:81,
2.2:81,
3:27,
4:135,
5:81,
6:27,
7:135,
8.1:81,
8.2:81,
9:81
}

In [12]:
data['coord.final.num'] = np.where(data['Zona Final'] == 'volver',None, data['Zona Final']).astype('float')


In [13]:
data['Zona inicial x'] = data['Zona inicial'].map(map_zona_coord_x)
data['Zona Final x'] = data['coord.final.num'].map(map_zona_coord_x)
data['Zona inicial y'] = data['Zona inicial'].map(map_zona_coord_y)
data['Zona Final y'] = data['coord.final.num'].map(map_zona_coord_y)
data.iloc[:,-5:]

,coord.final.num,Zona inicial x,Zona Final x,Zona inicial y,Zona Final y
0,3.0,136,45,27,27
1,6.0,255,136,81,27
2,8.1,227,255,135,81
3,7.0,255,227,81,135
4,8.1,207,255,81,81
5,6.0,136,136,81,27
6,5.0,136,136,135,81
7,1.0,30,45,81,135
8,2.1,30,70,81,81
9,4.0,136,136,81,135


In [14]:
# pasar en ambos formatos. Crear las dos cols originales q sean strings

data['coord.inicial'] = round(data['Zona inicial x'],0).astype('str').str.split('.').str[0]+','+round(data['Zona inicial y'],0).astype('str').str.split('.').str[0]
data['coord.final'] = round(data['Zona Final x'],0).astype('str').str.split('.').str[0]+','+round(data['Zona Final y'],0).astype('str').str.split('.').str[0]


In [15]:
data['coord.inicial'] = np.where(data['coord.inicial']=='nan,nan', None, data['coord.inicial'])
data['coord.final'] = np.where(data['coord.final']=='nan,nan', None, data['coord.final'])

In [16]:
data[['coord.inicial','coord.final']]

,coord.inicial,coord.final
0,"136,27","45,27"
1,"255,81","136,27"
2,"227,135","255,81"
3,"255,81","227,135"
4,"207,81","255,81"
5,"136,81","136,27"
6,"136,135","136,81"
7,"30,81","45,135"
8,"30,81","70,81"
9,"136,81","136,135"


In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Fecha                52 non-null     datetime64[ns]
 1   etiquetado           52 non-null     object        
 2   name                 52 non-null     object        
 3   Session Start        52 non-null     object        
 4   Session End          52 non-null     object        
 5   Evento               52 non-null     object        
 6   Tag Notes            0 non-null      float64       
 7   Tag Start            52 non-null     object        
 8   Tag End              52 non-null     object        
 9   Tag Duration (secs)  52 non-null     float64       
 10  Categoria            52 non-null     object        
 11  Zona inicial         52 non-null     object        
 12  Zona Final           52 non-null     object        
 13  Accion               52 non-null     

In [18]:
data['Temporada'] = 'Temp 02-2022'
data['partido'] = 'Independiente'

In [73]:
data.to_excel("../Datos generados/reserva02-22-fecha4_independiente_situaciones.xlsx",index=False)